# Wakeword Training (CUDA GPU Only + TorchMetrics + TensorBoard)

Bu notebook, mevcut eğitim pipeline'ınızı **yalnızca GPU üzerinde** çalıştırır, her 10 epoch'ta bir checkpoint kaydeder, TorchMetrics ile metrikleri hesaplar ve TensorBoard ile canlı olarak izlenebilir loglar üretir.

**Önemli:** CPU eğitimi yasaklanmıştır. CUDA GPU zorunludur.

In [1]:
# Ortam ve bağımlılık kontrolü (CUDA GPU zorunlu)
import sys, os
import torch

# CUDA kontrolü - CPU eğitimi yasak
if not torch.cuda.is_available():
    raise RuntimeError("CUDA GPU gerekli. CPU eğitimi politikaya göre devre dışı.")

print(f"Python: {sys.version}")
print(f"Torch: {torch.__version__}")
print(f"CUDA kullanılabilir: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

# CUDA optimizasyonları
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")  # TF32 etkinleştir
except Exception:
    pass

device = torch.device('cuda')  # GPU zorunlu
print(f"Kullanılan cihaz: {device}")

Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Torch: 2.1.2+cu118
CUDA kullanılabilir: True
GPU: NVIDIA GeForce RTX 3060 Ti
Kullanılan cihaz: cuda


## Kütüphaneler ve proje modülleri

In [ ]:
# Proje modüllerini içe aktar
import sys, os
sys.path.append(os.getcwd())

from training.enhanced_dataset import EnhancedWakewordDataset, EnhancedAudioConfig, create_dataloaders
from training.feature_extractor import FeatureExtractor

# TorchMetrics & TensorBoard (sklearn.metrics kullanarak gradio_app ile uyumlu)
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import numpy as np
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration Classes (gradio_app ile aynı)
class AudioConfig:
    SAMPLE_RATE = 16000
    DURATION = 1.7
    N_MELS = 80
    N_FFT = 2048
    HOP_LENGTH = 512
    WIN_LENGTH = 2048
    FMIN = 0
    FMAX = 8000

# Model Configuration
class ModelConfig:
    def __init__(self, input_size=128):
        self.HIDDEN_SIZE = 256
        self.NUM_LAYERS = 2
        self.DROPOUT = 0.6
        self.NUM_CLASSES = 2
        self.INPUT_SIZE = input_size  # Mel spectrogram height (frequency bins)

class TrainingConfig:
    BATCH_SIZE = 32
    LEARNING_RATE = 0.0001
    EPOCHS = 100
    VALIDATION_SPLIT = 0.2

# Enhanced Wakeword Model (gradio_app'den uyarlanmış)
class EnhancedWakewordModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # CNN layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout_cnn = nn.Dropout(0.3)

        # LSTM input size calculation
        # After 3 conv layers with maxpool: input_size // 8
        lstm_input_size = 128 * (self.config.INPUT_SIZE // 8)

        self.lstm = nn.LSTM(
            input_size=lstm_input_size,
            hidden_size=self.config.HIDDEN_SIZE,
            num_layers=self.config.NUM_LAYERS,
            batch_first=True,
            dropout=self.config.DROPOUT if self.config.NUM_LAYERS > 1 else 0,
            bidirectional=True,
        )
        self.attention = nn.Linear(self.config.HIDDEN_SIZE * 2, 1)
        self.dropout_lstm = nn.Dropout(self.config.DROPOUT)
        self.fc1 = nn.Linear(self.config.HIDDEN_SIZE * 2, 128)
        self.fc2 = nn.Linear(128, self.config.NUM_CLASSES)

    def forward(self, x):
        # x: (B, C, T) - mel spectrogram
        x = x.unsqueeze(1)  # (B, 1, C, T)

        # CNN feature extraction
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = self.dropout_cnn(x)  # (B, 128, H', W')

        B, C, Hp, Wp = x.shape
        # LSTM için zaman boyutu W', feature boyutu 128*H'
        x = x.permute(0, 3, 1, 2).contiguous()  # (B, W', 128, H')
        x = x.view(B, Wp, C * Hp)               # (B, W', 128*H')

        # LSTM processing
        lstm_out, _ = self.lstm(x)              # (B, W', 2*hidden)

        # Attention mechanism
        attn = torch.softmax(self.attention(lstm_out), dim=1)  # (B, W', 1)
        attended = torch.sum(attn * lstm_out, dim=1)           # (B, 2*hidden)

        # Classification head
        out = self.dropout_lstm(attended)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

## DataLoader ve eğitim yapılandırması

In [3]:
from datetime import datetime

# Mevcut dizin yapısını kullan
POS_DIR = 'positive_dataset'      # Pozitif wakeword örnekleri
NEG_DIR = 'negative_dataset'      # Negatif örnekler
FEATURES_DIR = 'features/train'   # Hazır .npy feature'lar
BACKGROUND_DIR = 'background_noise'  # Arka plan gürültüsü
RIRS_DIR = 'datasets/mit_rirs/rir_data'  # RIR verileri

# Hiperparametreler (gradio_app ile aynı)
batch_size = TrainingConfig.BATCH_SIZE
lr = TrainingConfig.LEARNING_RATE
epochs = 50  # Notebook için daha kısa
val_split = TrainingConfig.VALIDATION_SPLIT

# Audio konfigürasyonu
audio_cfg = EnhancedAudioConfig(
    use_precomputed_features=True,
    features_dir=FEATURES_DIR,
    use_rirs_augmentation=False,  # Basit tutmak için devre dışı
    rirs_dataset_path=RIRS_DIR,
)

# DataLoader'ları oluştur
print("DataLoader'lar oluşturuluyor...")
try:
    train_loader, val_loader = create_dataloaders(
        positive_dir=POS_DIR,
        negative_dir=NEG_DIR,
        features_dir=FEATURES_DIR,
        rirs_dir=RIRS_DIR,
        batch_size=batch_size,
        config=audio_cfg,
    )
    print(f"✅ Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
except Exception as e:
    print(f"❌ DataLoader oluşturma hatası: {e}")
    print("Feature dosyalarının varlığını kontrol edin...")
    raise

NameError: name 'EnhancedAudioConfig' is not defined

## Model, optimizer, TorchMetrics ve TensorBoard

In [ ]:
# Model ve optimizasyon (CUDA GPU zorunlu)
device = torch.device('cuda')  # GPU zorunlu

# Veri örneğinden giriş kanal sayısını türet
sample_batch = next(iter(train_loader))
derived_input_size = int(sample_batch['features'].shape[1])  # (B, C, T) -> C
print(f"Feature input size: {derived_input_size}")

# Model konfigürasyonu
model_cfg = ModelConfig(input_size=derived_input_size)

# Model oluştur
model = EnhancedWakewordModel(model_cfg).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

# TorchMetrics (sklearn.metrics kullanarak gradio_app ile uyumlu)
# Metrikler eğitim sırasında hesaplanacak

# TensorBoard setup
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
run_name = datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join('runs', f'wakeword-{run_name}')
writer = SummaryWriter(log_dir)

print(f"✅ Model oluşturuldu: {sum(p.numel() for p in model.parameters()):,} parametre")
print(f"✅ TensorBoard log dizini: {log_dir}")
print(f"✅ Cihaz: {device}")

## Eğitim döngüsü (10 epoch’ta checkpoint + resume)

In [ ]:
import math
from pathlib import Path

ckpt_dir = Path('checkpoints')
ckpt_dir.mkdir(exist_ok=True)
last_ckpt_path = ckpt_dir / 'last_checkpoint.pth'
best_ckpt_path = ckpt_dir / 'best_model.pth'

start_epoch = 0
best_val_f1 = -1.0

# Resume logic
if last_ckpt_path.exists():
    state = torch.load(last_ckpt_path, map_location=device)
    model.load_state_dict(state['model_state'])
    optimizer.load_state_dict(state['optimizer_state'])
    start_epoch = state.get('epoch', 0) + 1
    best_val_f1 = state.get('best_val_f1', -1.0)
    print(f"📁 Checkpoint'ten devam: epoch {start_epoch}, best_val_f1={best_val_f1:.4f}")

# Metrik sıfırlama fonksiyonu (sklearn için gerekli değil)
def step_metrics_reset():
    pass  # sklearn metrics için sıfırlama gerekmez

# Değerlendirme fonksiyonu (sklearn.metrics kullanarak)
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating", leave=False):
            x = batch['features'].to(device)
            y = batch['label'].to(device)

            logits = model(x)
            loss = criterion(logits, y)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)[:, 1]
            preds = (probs > 0.5).long()  # 0.5 threshold ile binary classification

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    n = max(1, len(loader))

    # sklearn ile metrikleri hesapla
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='binary', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)

    return {
        'loss': total_loss / n,
        'acc': acc,
        'prec': prec,
        'rec': rec,
        'f1': f1,
    }

# Checkpoint kaydetme
def save_checkpoint(epoch, tag=None):
    tag_suffix = f"_epoch_{epoch:04d}" if tag is None else f"_{tag}"
    path = ckpt_dir / f"checkpoint{tag_suffix}.pth"
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'best_val_f1': best_val_f1,
        'config': model_cfg.__dict__,
    }, path)
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'best_val_f1': best_val_f1,
        'config': model_cfg.__dict__,
    }, last_ckpt_path)
    print(f"💾 Checkpoint kaydedildi: {path}")

# Eğitim döngüsü
print("🚀 Eğitim başlatılıyor...")
print(f"   Epochs: {epochs}, Batch size: {batch_size}, Learning rate: {lr}")
print(f"   Cihaz: {device}")

for epoch in range(start_epoch, epochs):
    model.train()
    step_metrics_reset()
    running_loss = 0.0
    train_preds = []
    train_labels = []

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for i, batch in enumerate(train_pbar):
        x = batch['features'].to(device)
        y = batch['label'].to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        probs = torch.softmax(logits, dim=1)[:, 1]
        preds = (probs > 0.5).long()  # 0.5 threshold ile binary classification

        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(y.cpu().numpy())

        # Progress bar güncelle
        train_pbar.set_postfix({
            'loss': f"{running_loss/(i+1):.4f}",
        })

    # Eğitim metrikleri (sklearn ile)
    n_train = max(1, len(train_loader))
    train_loss = running_loss / n_train
    train_acc = accuracy_score(train_labels, train_preds)
    train_prec = precision_score(train_labels, train_preds, average='binary', zero_division=0)
    train_rec = recall_score(train_labels, train_preds, average='binary', zero_division=0)
    train_f1 = f1_score(train_labels, train_preds, average='binary', zero_division=0)

    # Validasyon
    val_stats = evaluate(model, val_loader)

    # TensorBoard logging
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_stats['loss'], epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/val', val_stats['acc'], epoch)
    writer.add_scalar('F1/train', train_f1, epoch)
    writer.add_scalar('F1/val', val_stats['f1'], epoch)
    writer.add_scalar('Precision/train', train_prec, epoch)
    writer.add_scalar('Precision/val', val_stats['prec'], epoch)
    writer.add_scalar('Recall/train', train_rec, epoch)
    writer.add_scalar('Recall/val', val_stats['rec'], epoch)

    # Progress print
    print(f"Epoch {epoch+1:3d}/{epochs} | "
          f"train_loss={train_loss:.4f} val_loss={val_stats['loss']:.4f} | "
          f"train_f1={train_f1:.4f} val_f1={val_stats['f1']:.4f}")

    # Best model kaydet
    if val_stats['f1'] > best_val_f1:
        best_val_f1 = val_stats['f1']
        torch.save({
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'best_val_f1': best_val_f1,
            'config': model_cfg.__dict__,
        }, best_ckpt_path)
        print(f"🏆 Best model güncellendi: F1={best_val_f1:.4f}")

    # 10 epoch'ta bir checkpoint
    if (epoch + 1) % 10 == 0:
        save_checkpoint(epoch + 1)

# Final checkpoint
save_checkpoint(epochs, tag='final')
writer.close()
print('✅ Eğitim tamamlandı!')
print(f"📊 Best validation F1: {best_val_f1:.4f}")
print(f"📁 TensorBoard logs: {log_dir}")

## TensorBoard İzleme

Eğitim metriklerini canlı olarak izlemek için TensorBoard'ı başlatın:

### Terminal Komutu:
```powershell
# PowerShell'de (Windows)
$env:LOGDIR="runs"; tensorboard --logdir $env:LOGDIR --host 127.0.0.1 --port 6006
```

### Alternatif olarak VS Code terminalinde:
```bash
# Linux/Mac veya WSL
LOGDIR=runs tensorboard --logdir $LOGDIR --host 127.0.0.1 --port 6006
```

### Tarayıcıda erişim:
TensorBoard başladıktan sonra: http://127.0.0.1:6006

### İzlenecek Metrikler:
- **Loss**: Eğitim ve validasyon kaybı
- **Accuracy**: Doğruluk oranları  
- **F1-Score**: F1 skorları (precision + recall harmonik ortalaması)
- **Precision**: Kesinlik
- **Recall**: Duyarlılık

In [ ]:
# Eğitim sonrası model değerlendirmesi
if os.path.exists('best_wakeword_model.pth'):
    print("🎯 Eğitim tamamlandı! Model değerlendirmesi için gradio_app.py'yi kullanabilirsiniz.")
    print("📁 Kaydedilen dosyalar:")
    print("   - best_wakeword_model.pth (en iyi model)")
    print("   - checkpoints/last_checkpoint.pth (son checkpoint)")
    print("   - runs/ dizininde TensorBoard logları")
else:
    print("❌ Model dosyası bulunamadı. Eğitimi tekrar kontrol edin.")

## Sonraki Adımlar

1. **TensorBoard ile Metrik İzleme**: Eğitim metriklerini canlı olarak izleyin
2. **Model Değerlendirmesi**: `gradio_app.py` ile model performansını test edin
3. **Deployment**: En iyi modeli `wakeword_deployment_model.pth` olarak kaydedin
4. **İnce Ayar**: Gerekirse hiperparametreleri ayarlayıp eğitimi tekrarlayın

### Kullanılan Paket Sürümleri (gradio_app.py ile aynı):
- **PyTorch**: 2.1.2+cu118 (CUDA 11.8)
- **TorchAudio**: 2.1.2+cu118
- **TorchVision**: 0.16.2+cu118
- **Scikit-learn**: >=1.0.0 (metrikler için)
- **TensorBoard**: >=2.14.0
- **CUDA GPU**: Zorunlu (CPU eğitimi yasak)

*Not: TorchMetrics yerine sklearn.metrics kullanılıyor (gradio_app uyumluluğu için)*

### Diz Yapısı:
```
├── positive_dataset/     # Wakeword örnekleri
├── negative_dataset/     # Negatif örnekler
├── features/train/       # Hazır .npy feature'lar
├── background_noise/     # Arka plan gürültüsü
├── checkpoints/          # Checkpoint'lar
├── runs/                 # TensorBoard logları
└── best_wakeword_model.pth  # En iyi model
```

## Model Evaluation (Validation Set)

In [ ]:
# Load best model for evaluation on validation set
if os.path.exists('best_wakeword_model.pth'):
    checkpoint = torch.load('best_wakeword_model.pth', map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    print(f"✅ Best model loaded (epoch {checkpoint['epoch'] + 1}, best_val_f1: {checkpoint['best_val_f1']:.4f})")

    # Evaluate on validation set (since we don't have a separate test set)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating on validation set"):
            x = batch['features'].to(device)
            y = batch['label'].to(device)

            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:, 1]
            preds = (probs > 0.5).long()  # 0.5 threshold

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)

    print(f"\n📊 Validation Set Performance:")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall: {recall:.4f}")
    print(f"   F1-Score: {f1:.4f}")

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negative', 'Wakeword'],
                yticklabels=['Negative', 'Wakeword'])
    plt.title('Confusion Matrix (Validation Set)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Classification report
    print("\n📋 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Negative', 'Wakeword']))

else:
    print("❌ No trained model found. Please run the training cells first.")

## Model Inference

In [ ]:
def predict_wakeword_from_features(features, model, device, threshold=0.5):
    """Predict if audio features contain wakeword"""
    model.eval()

    # Convert to tensor and add batch dimension
    if not isinstance(features, torch.Tensor):
        features = torch.FloatTensor(features)

    # Ensure correct shape (B, C, T)
    if features.dim() == 2:
        features = features.unsqueeze(0)  # Add batch dimension

    features = features.to(device)

    # Make prediction
    with torch.no_grad():
        logits = model(features)
        probs = torch.softmax(logits, dim=1)
        wakeword_prob = probs[0][1].item()

    is_wakeword = wakeword_prob >= threshold
    return is_wakeword, wakeword_prob

# Test prediction function on test files
if os.path.exists('best_wakeword_model.pth'):
    checkpoint = torch.load('best_wakeword_model.pth', map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    print("🎯 Wakeword Detection System Ready!")
    print("Testing on sample files from test_files/ directory...")

    # Test on available test files
    test_files = ['test_files/0.wav', 'test_files/1.wav', 'test_files/2.wav', 'test_files/3.wav']

    for test_file in test_files:
        if os.path.exists(test_file):
            try:
                # Load and process audio file
                audio, sr = sf.read(test_file)
                if sr != AudioConfig.SAMPLE_RATE:
                    # Resample if needed
                    audio = librosa.resample(audio, orig_sr=sr, target_sr=AudioConfig.SAMPLE_RATE)

                # Extract features (simple mel spectrogram)
                mel_spec = librosa.feature.melspectrogram(
                    y=audio,
                    sr=AudioConfig.SAMPLE_RATE,
                    n_mels=AudioConfig.N_MELS,
                    n_fft=AudioConfig.N_FFT,
                    hop_length=AudioConfig.HOP_LENGTH,
                    fmin=AudioConfig.FMIN,
                    fmax=AudioConfig.FMAX
                )
                mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

                # Normalize
                mel_spec = (mel_spec - mel_spec.mean()) / (mel_spec.std() + 1e-8)

                # Predict
                is_wakeword, confidence = predict_wakeword_from_features(mel_spec, model, device)

                print(f"\n📁 Testing: {test_file}")
                print(f"   Wakeword detected: {is_wakeword}")
                print(f"   Confidence: {confidence:.3f}")
                print(f"   Threshold: 0.50")

            except Exception as e:
                print(f"❌ Error processing {test_file}: {e}")
        else:
            print(f"⚠️  Test file not found: {test_file}")
else:
    print("❌ Model not trained yet. Please run training cells first.")


## Save Final Model

In [ ]:
# Save the complete model for deployment
if os.path.exists('best_wakeword_model.pth'):
    # Load the checkpoint to get training info
    checkpoint = torch.load('best_wakeword_model.pth', map_location=device)

    # Create a complete deployment package
    deployment_package = {
        'model_state_dict': checkpoint['model_state'],
        'model_config': checkpoint.get('config', {
            'HIDDEN_SIZE': ModelConfig.HIDDEN_SIZE,
            'NUM_LAYERS': ModelConfig.NUM_LAYERS,
            'DROPOUT': ModelConfig.DROPOUT,
            'NUM_CLASSES': ModelConfig.NUM_CLASSES,
            'input_size': derived_input_size
        }),
        'audio_config': {
            'SAMPLE_RATE': AudioConfig.SAMPLE_RATE,
            'DURATION': AudioConfig.DURATION,
            'N_MELS': AudioConfig.N_MELS,
            'N_FFT': AudioConfig.N_FFT,
            'HOP_LENGTH': AudioConfig.HOP_LENGTH,
            'FMIN': AudioConfig.FMIN,
            'FMAX': AudioConfig.FMAX
        },
        'training_info': {
            'best_val_f1': checkpoint.get('best_val_f1', 0),
            'epoch': checkpoint.get('epoch', 0) + 1,
            'device': str(device)
        },
        'classes': ['negative', 'wakeword']
    }

    # Save deployment package
    torch.save(deployment_package, 'wakeword_deployment_model.pth')
    print("✅ Deployment model saved as 'wakeword_deployment_model.pth'")

    # Save model architecture for reference
    with open('model_architecture.txt', 'w') as f:
        f.write("Wakeword Detection Model Architecture\n")
        f.write("================================\n\n")
        f.write("Model Type: CNN + LSTM with Attention\n")
        f.write(f"Input Shape: ({deployment_package['model_config']['input_size']}, T)\n")
        f.write(f"Hidden Size: {deployment_package['model_config']['HIDDEN_SIZE']}\n")
        f.write(f"Number of Layers: {deployment_package['model_config']['NUM_LAYERS']}\n")
        f.write(f"Dropout: {deployment_package['model_config']['DROPOUT']}\n")
        f.write(f"Number of Classes: {deployment_package['model_config']['NUM_CLASSES']}\n")

        # Create a temporary model to count parameters
        temp_model = EnhancedWakewordModel(ModelConfig())
        temp_model.cfg = deployment_package['model_config']
        f.write(f"Parameters: {sum(p.numel() for p in temp_model.parameters()):,}\n")
        f.write(f"Device: {device}\n")
        f.write(f"Best Validation F1: {deployment_package['training_info']['best_val_f1']:.4f}\n")

    print("✅ Model architecture saved as 'model_architecture.txt'")

    print("\n🎉 Model deployment package ready!")
    print("Files created:")
    print("   - wakeword_deployment_model.pth (complete model)")
    print("   - model_architecture.txt (model specs)")
    print("   - best_wakeword_model.pth (training checkpoint)")

else:
    print("❌ No trained model found to save.")

## System Summary

In [ ]:
print(f"\n🚀 Next Steps:")
print(f"   1. Add your wakeword recordings to positive_dataset/")
print(f"   2. Add negative samples to negative_dataset/")
print(f"   3. Add background noise to background_noise/")
print(f"   4. Run the training cells above")
print(f"   5. Use the trained model for wakeword detection")